# Case study 1: Simulation with a SARS-CoV-2 Model

In [9]:
!pip install git+https://github.com/y0-causal-inference/eliater.git 

  Cloning https://github.com/y0-causal-inference/eliater.git to /private/var/folders/fs/kx46_43x04ndj3yryggvkg5r0000gn/T/pip-req-build-5iigqke7
  Running command git clone --quiet https://github.com/y0-causal-inference/eliater.git /private/var/folders/fs/kx46_43x04ndj3yryggvkg5r0000gn/T/pip-req-build-5iigqke7
  Resolved https://github.com/y0-causal-inference/eliater.git to commit 722814a686ef4771548cdbb0b9c59804014438c5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [10]:
import numpy as np

# from eliater import version_df
from eliater.discover_latent_nodes import find_nuisance_variables, remove_nuisance_variables
from src.eliater.examples.sars_cov2 import sars_large_example as example
from src.eliater.examples.sars_cov2 import generate_continuous
from eliater.network_validation import print_graph_falsifications
from y0.algorithm.estimation import estimate_ace
from y0.algorithm.identify import Identification, identify_outcomes
from y0.dsl import P, Variable

# version_df()

In [45]:
treatment = Variable("EGFR")
outcome = Variable("cytok")
SEED = 100

This is case study 1 in Figure 6 in this paper: Eliater: an open source software for causal query estimation from observational measurements of biomolecular networks. The figure below shows the SARS-CoV-2 network (Mohammad-Taheri et al., 2022; Zucker
et al., 2021), which models the activation of Cytokine Release Syndrome (Cytokine Storm), a known factor causing tissue damage in severely ill SARS-CoV-2 patients (Ulhaq and Soraya, 2020).

In [46]:
graph = example.graph

This case study used synthetic observational data. The generation of this synthetic data was inspired by common biological practices. For each endogenous variable $X$ including $EGFR$, we represented  biomolecular reactions using Hill equations \cite{alon2019introduction}. Specifically, we generated observations of each node $X$ from a Binomial distribution with probability of $\frac{1}{1 + \exp(\mathbf{\theta}^{\prime} Pa(X) + \theta_0)}$, where $Pa(X)$ is a $q \times 1$ vector of measurements related to the parent of $X$, $\mathbf{\theta}'$ is a $1 \times q$ parameter vector, and $\theta_0$ is a scalar. The probability distributions for the exogenous variables were simulated from a Binomial distribution with a random probability between 0.4 to 0.8.

In [47]:
# get observational data
#data = example.generate_data(1000, seed=SEED)
data = example.generate_data(1000, seed=SEED)
data.head()

,SARS_COV2,ACE2,Ang,AGTR1,ADAM17,Toci,Sil6r,EGF,TNF,Gefi,EGFR,PRR,NFKB,IL6STAT3,IL6AMP,cytok
0,69.710048,20.806396,93.515972,98.625068,99.757369,45.055621,79.315974,95.263883,99.873655,36.980686,1,96.551967,34.955189,67.052502,67.583769,89.418508
1,51.613370,35.918803,72.495854,99.604241,99.184755,51.214968,67.850258,97.598495,99.232373,31.756410,1,87.885129,32.950568,22.762004,33.443954,52.667406
2,38.597698,48.446329,37.170298,80.791525,98.023349,45.981240,86.907642,97.275595,99.890245,42.516384,1,83.560508,29.187536,41.223031,47.814307,71.118222
3,61.607576,28.396223,85.190953,99.023122,100.925275,63.157239,61.027930,96.234067,100.188550,49.204452,1,93.296112,39.499817,8.329443,27.935436,43.480753
4,75.863532,16.175534,95.552035,99.389911,98.459049,42.702804,91.807464,94.987621,98.913570,56.360465,1,98.137536,30.646429,60.188676,59.629591,84.846896


## Step 1: Verify correctness of the network structure

In [58]:
print_graph_falsifications(
    graph=graph, data=data, method="chi-square", verbose=True, significance_level=0.01
)

array([[2., 0., 2., ..., 2., 2., 2.],
       [1., 1., 2., ..., 0., 0., 0.],
       [0., 2., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 2., ..., 1., 1., 1.],
       [2., 0., 2., ..., 1., 1., 1.],
       [2., 0., 2., ..., 1., 1., 1.]])

None of the tests failed for this graph. The reason is that the data is simulated according to the graph structure, hence we expect the d-separations implied by the network be validated by the data. 

## Step 2: Check query identifiability

In [18]:
identify_outcomes(graph=graph, treatments=treatment, outcomes=outcome)

Sum[ACE2, ADAM17, AGTR1, Ang, IL6AMP, IL6STAT3, NFKB, PRR, SARS_COV2, Sil6r, TNF, Toci](P(ACE2 | SARS_COV2) * P(AGTR1 | ACE2, Ang, SARS_COV2) * P(IL6AMP | ACE2, ADAM17, AGTR1, Ang, EGF, EGFR, Gefi, IL6STAT3, NFKB, PRR, SARS_COV2, Sil6r, TNF, Toci) * P(IL6STAT3 | ACE2, ADAM17, AGTR1, Ang, SARS_COV2, Sil6r, Toci) * P(TNF | ACE2, ADAM17, AGTR1, Ang, SARS_COV2) * P(cytok | ACE2, ADAM17, AGTR1, Ang, EGF, EGFR, Gefi, IL6AMP, IL6STAT3, NFKB, PRR, SARS_COV2, Sil6r, TNF, Toci) * Sum[ACE2, ADAM17, AGTR1, Ang, EGF, EGFR, Gefi, IL6AMP, IL6STAT3, NFKB, PRR, SARS_COV2, Sil6r, TNF, cytok](P(ACE2, ADAM17, AGTR1, Ang, EGF, EGFR, Gefi, IL6AMP, IL6STAT3, NFKB, PRR, SARS_COV2, Sil6r, TNF, Toci, cytok)) * P(ADAM17 | ACE2, AGTR1, Ang, SARS_COV2, Toci) * P(Sil6r | ACE2, ADAM17, AGTR1, Ang, SARS_COV2, Toci) * P(Ang | ACE2, SARS_COV2) * P(SARS_COV2) * P(NFKB | ACE2, ADAM17, AGTR1, Ang, EGF, EGFR, Gefi, PRR, SARS_COV2, TNF) * P(PRR | ACE2, Gefi, SARS_COV2))

The query is identifiable.

## Step 3: Find nuisance variables and mark them as latent

This function finds the nuisance variables for the input graph.

In [19]:
nuisance_variables = find_nuisance_variables(graph, treatments=treatment, outcomes=outcome)
nuisance_variables

set()

No variable is identified as the nuisance variable. Hence the simplified network in the next step will produce a graph similar to the original graph.

## Step 4: Simplify the network

The following function finds the nuisance variables (step 3), marks them as latent and then applies Evan's simplification rules to remove the nuisance variables. As there are no nuisance variables, the new graph will be the same as the original graph.

In [48]:
new_graph = remove_nuisance_variables(graph, treatments=treatment, outcomes=outcome)

## Step 5: Estimate the query

In [49]:
estimate_ace(new_graph, treatments=treatment, outcomes=outcome, data=data)

0.7826734394202646

## Evaluation criterion
As we used synthetic data set, we were able to generate two interventional data sets where in
one EGFR was set to 1, and the other one EGFR is set to 0. The ATE was calculated by subtracting the average value of Cytokine Storm obtained from each interventional data,
resulting in the ground truth ATE=-0.018. The negative ATE indicates that the Gefitinib drug can reduce the increase in Cytokine Storm levels, hence can help in treating patients having SARS-Cov-2.

In [50]:
def get_background_ace(seed=None) -> float:
    # get interventional data where treatment is set to 1
    data_1 = generate_continuous(1000, {treatment: 1.0}, seed=seed)
    # get interventional data where treatment is set to 0
    data_0 = generate_continuous(1000, {treatment: 0.0}, seed=seed)
    return data_1.mean()[outcome.name] - data_0.mean()[outcome.name]

# get the true value of ATE
get_background_ace(seed=SEED)

0.7851941596613301

In [82]:
# def get_background_ace(seed=None) -> float:
#     # get interventional data where treatment is set to 1
#     data_1 = example.generate_data(1000, {treatment: 1.0}, seed=seed)
#     # get interventional data where treatment is set to 0
#     data_0 = example.generate_data(1000, {treatment: 0.0}, seed=seed)
#     return data_1.mean()[outcome.name] - data_0.mean()[outcome.name]
# 
# # get the true value of ATE
# get_background_ace(seed=SEED)

-0.024999999999999967

jThe estimated $\widehat{\mathrm{ATE}}=-0.2$, comparable in sign and magnitute to the ground truth ATE=-0.02. The discrepancy in the value of results is due to non-linear, and complex data generation procedure, which is similar to the real-life experimental artifacts, and due to the approximate nature of the modeling assumption.

In [65]:
#Relative change
((-0.22+0.018)/-0.018)

1122.2222222222224